# Data Wrangling: Part 2 Provider Group Matches with Associated Hospitals

## Purpose:
The purpose of this notebook is to clean the data we acquired from the prior notebook. Specifically, we will be taking a look at provider groups in this notebook and attempting to match them with their associated hospitals.

Lets first import some of the basic packages we will be using. All of these packages are availible with !pip install or conda install.

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl
import glob
from ast import literal_eval
from collections import Counter

### Files we need to obtain

We need to first obtain the files that have our provider groups. I have stored the parent directory in a dotenv locally, but feel free to create your own variables to replace the ones here. These should be locations where you stored the prior notebook items.

In [2]:
load_dotenv()

hyperlink_path = 'json_completed_hyperlinks_update.csv'
parent_dir = os.getenv('dir')
data_dir = os.path.join(parent_dir,'data_update')

df = pd.read_csv(hyperlink_path, header=None)
df.head()
df.columns = ['ParseID','Hyperlink']
hyperlinks = df['Hyperlink'].tolist()

def foldername(hyperlink):
    hyperlink = hyperlink.split('/')[-1]
    return hyperlink[0:-8]
def providers_path(folder):
    return os.path.join(data_dir,folder,folder+'_providers.csv')

folder_names= [foldername(hyperlink) for hyperlink in hyperlinks]
provider_files = [providers_path(folder_name) for folder_name in folder_names]

provider_files[0]

'D:/Vignesh/Capstone\\data_update\\2023-01-01_ALL-SAVERS-INSURANCE-COMPANY_Insurer_PPO---NDC_PPO-NDC_in-network-rates\\2023-01-01_ALL-SAVERS-INSURANCE-COMPANY_Insurer_PPO---NDC_PPO-NDC_in-network-rates_providers.csv'

### Lets read one:

Let us read one file to see how the files are formated and become fimilar with the data.

In [3]:
df = pd.read_csv(provider_files[0], usecols=['tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval})
folder = folder_names[0]

In [4]:
df.dtypes
df.head()

,tin,npi_provider_groups
0,593582520,[1225090087]
1,272050459,[1639508567]
2,160743209,[1609314343]
3,561844651,"[1215134309, 1043649635, 1851721047, 1285715185]"
4,371756970,[1174710636]


Notice how we have an array of npi numbers, lets explode this.

In [5]:
df_explode = df.explode(column= 'npi_provider_groups',ignore_index=True)
df_explode.head()

,tin,npi_provider_groups
0,593582520,1225090087
1,272050459,1639508567
2,160743209,1609314343
3,561844651,1215134309
4,561844651,1043649635


### Lets add all the files to a single dataframe

Since the providers files are relatively small, we will be combining all the providers group into one list with the follow code. This may take several minutes to execute depending on how many files were parsed.

In [6]:
df = pd.concat((pd.read_csv(f, usecols=['tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval}) for f in provider_files), ignore_index=True)
df.info()

Lets check to see if there are any NaN values.

In [7]:
df[df.isna().any(axis=1)]

,tin,npi_provider_groups
1993,NaN,"[1487195756, 1942589122]"
21891,NaN,[]
62098,NaN,[]
101913,NaN,[]
141728,NaN,[]
...,...,...
17151887,NaN,[]
17191702,NaN,[]
17231517,NaN,[]
17271332,NaN,[]


TIN is a tax identification number and is an option to organize provider groups outside of their reference numbers. This is useful because reference numbers are specific per json file and do not correlate with other json files, but TIN numbers do. 

For example, the first json file my have the reference number 1 = NPI 11111 but the second json file might have reference number 1 = NPI 22222. We do not want to treat these as equal. however maybe the first json file has reference number 3 = 22222. These two TIN should match. There are instances where NPIs may match to many TINs. This would be if a physician works for hospital and maybe has a private practice as well. I will get more into this later.

For now, lets drop na values in either column as there will be no way to figure out which hospital these groups belong to, if they belong to any group at all. Lets also drop duplicats along TIN values, as they should have matching NPI value arrays.

In [8]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['tin'], inplace=True, ignore_index=True)

Lets take a look at the data.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46854 entries, 0 to 46853
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tin                  46854 non-null  float64
 1   npi_provider_groups  46854 non-null  object 
dtypes: float64(1), object(1)
memory usage: 732.2+ KB


It may be easier to work with the data in exploded form in the future, but for now I have this section commented out.

In [10]:
# df = df.explode(column= 'npi_provider_groups',ignore_index=True)

### Matching Providers with Hospitals

The following file is provided by the CMS. It is an associated hospital affilcation for NPI numbers in CSV format. CCN is a unquie number that refers to the facility they work for or are affilated with. Hospitals can have parent CCNs, this may be the case for large hospital systems.

In [11]:
npi_path = 'Facility_Affiliation.csv'
npi = pd.read_csv(npi_path,usecols=['NPI','facility_afl_ccn','parent_ccn'], encoding='windows-1252')

In [12]:
npi.head()

,NPI,facility_afl_ccn,parent_ccn
0,1003000126,210003,NaN
1,1003000126,210022,NaN
2,1003000126,490063,NaN
3,1003000126,490145,NaN
4,1003000134,140010,NaN


Lets create a new column where we determine which CCN to use for each NPI. If parent_cnn is NaN, then we will use the facility_afl_ccn number. Below is a lambda function that can be applied to the dataframe to do that for us.

In [13]:
npi['ccn'] = npi.apply(lambda row: row['facility_afl_ccn'] if np.isnan(row['parent_ccn']) else row['parent_ccn'], axis=1)

I want to cast ccn as a string, as I am worried there may be letters the pop with facilities ccn.

In [14]:
npi=npi.astype({'ccn':'str'})
npi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1576038 entries, 0 to 1576037
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   NPI               1576038 non-null  int64  
 1   facility_afl_ccn  1576038 non-null  object 
 2   parent_ccn        5635 non-null     float64
 3   ccn               1576038 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 48.1+ MB


### Matching NPI groups with their hospital

The following can be implemented in several ways.

We are currently tasked with taking NPI groups, and matching them with their affilated hospital. As stated earlier, NPIs can be associated with many hospitals/facilities. Since we NPI's in an array form within our database, I believe the best way to do this is to take each NPI for a given TIN, do a majority cote of which hospital each NPI is associated with for a TIN, and match this to that hospital's CCN. I defined a function here that does exactly that and applies it to our dataframe. This may take some time.

In [15]:
def find_npi(x):
    ccns = []
    for npi_value in x['npi_provider_groups']:
        queried = npi[npi['NPI']==npi_value]
        lst = queried['ccn'].to_list()
        if lst: 
            ccns = ccns + lst
    if ccns:
        count= Counter(ccns)
        x['ccn'] = count.most_common()[0][0]
    else:
        x['ccn'] = None
    return x

df = df.apply(find_npi, axis=1)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46854 entries, 0 to 46853
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tin                  46854 non-null  float64
 1   npi_provider_groups  46854 non-null  object 
 2   ccn                  3895 non-null   object 
dtypes: float64(1), object(2)
memory usage: 1.1+ MB


Lets store this data. I used the to_parquet here, but you can easily store this as a csv as well. 

It seems like most of this data is not associated with hospitals. This makes sense given most NPIs are private practice or related to pharmacies.

In [17]:
df.to_parquet('tin_to_ccn.parquet')

Lets clean this data as well.

In [18]:
df.dropna(subset=['ccn'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3895 entries, 96 to 46853
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tin                  3895 non-null   float64
 1   npi_provider_groups  3895 non-null   object 
 2   ccn                  3895 non-null   object 
dtypes: float64(1), object(2)
memory usage: 121.7+ KB


In [19]:
df.head()

,tin,npi_provider_groups,ccn
96,1.356639e+09,[1356638811],297112
97,1.285698e+09,[1285698381],030101
106,1.508899e+09,[1508899253],290007
129,1.932107e+09,[1932106853],030055
131,1.215107e+09,[1215107347],060006


Lets store this data seperately, as this will be its useful form.

In [20]:
df.to_parquet('tin_to_ccn_nonan.parquet')